In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pathlib

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
tf.debugging.set_log_device_placement(False)

In [ ]:
!pip install -q git+https://github.com/tensorflow/docs

In [ ]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
dataset = pd.read_csv("/kaggle/input/pubg-finish-placement-prediction/train_V2.csv")[["damageDealt", "headshotKills", "killPlace","boosts", "heals", "winPlacePerc"]].dropna()



In [ ]:
train_dataset = dataset.sample(frac=0.9,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
train_stats = train_dataset.describe()
train_stats.pop("winPlacePerc")
train_stats = train_stats.transpose()
train_stats

In [ ]:
train_labels = train_dataset.pop("winPlacePerc")
test_labels = test_dataset.pop("winPlacePerc")

In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(len(train_dataset.keys()), )),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    


  model.compile(loss='mae',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model


In [ ]:
model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
EPOCHS = 2

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=1, batch_size=300,
  callbacks=[early_stop, tfdocs.modeling.EpochDots()])

In [ ]:
#loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)


In [ ]:
"""
test_predictions = model.predict(normed_test_data).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values')
plt.ylabel('Predictions')
lims = [0, 1]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)
"""

In [ ]:
"""
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error")
_ = plt.ylabel("Count")
"""

In [ ]:
test_set = pd.read_csv("/kaggle/input/pubg-finish-placement-prediction/test_V2.csv")[["Id","damageDealt", "headshotKills", "killPlace","boosts", "heals"]]

In [ ]:
test_set_id = test_set.pop("Id")
normed_test_set = norm(test_set)
predictions = model.predict(normed_test_set).flatten()

In [ ]:
with open("/kaggle/working/submission.csv", "w") as f:
    f.write("Id,winPlacePerc\n")
    for id, prediction in zip(test_set_id, predictions):
        f.write("{},{}\n".format(id, prediction))
        

In [ ]:
predict_df = pd.DataFrame(predictions, columns=["predictions"])

In [ ]:
pd.concat([test_set_id.to_frame(), predict_df], axis=1).to_csv("submission.csv", index=False)

In [ ]:
type(test_set_id)

In [ ]:
.